In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ipywidgets import interact

# Size of the 3D grid
N = 20

# Time step
dt = 0.05

# Initialize U and V
U = np.full((N, N, N), 0.0)  # Initial condition for U
V = np.zeros((N, N, N))  # Initial condition for V

# Initialize small area in center of V = 1
center_size = 2
V[N // 2 - center_size:N // 2 + center_size,
  N // 2 - center_size:N // 2 + center_size,
  N // 2 - center_size:N // 2 + center_size] = 1

# Parameters
Du, Dv = 1.0, 0.5  # Adjusted parameters for the coral pattern

# Initialize F and k
F_init = 0.0545
k_init = 0.062

# Function to compute F
def compute_F(F_init, x, y, z, t):
    # Compute the square distance from the center of the grid
    d_squared = (x - N/2)**2 + (y - N/2)**2 + (z - N/2)**2

    # Initial Gaussian-like function for the spatial distribution
    spatial_dependence = np.exp(-d_squared / (2.0 * (N/4)**2))

    # Time dependence: the concentration decreases at the center and increases in outer locations over time
    time_dependence = 1 + np.tanh((d_squared - (N*t/2000)**2)/(N**2/16))

    # Combine the spatial and temporal dependence
    F = F_init * spatial_dependence * time_dependence

    return F


# Function to compute k
def compute_k(k_init, x, y, z, t):
    # k is modulated by the distance from the origin (increasing with time), 
    # creating a moving wave pattern
    k = k_init + 0.0100 * np.sin(np.sqrt((x - t)**2 + (y - t)**2 + (z - t)**2))
    return k

# Function to calculate Laplacian using convolution
def laplacian(Z):
    laplacian_kernel = np.array([
        [[0.05, 0.2, 0.05],
         [0.2, -1, 0.2],
         [0.05, 0.2, 0.05]],

        [[0.05, 0.2, 0.05],
         [0.2, -1, 0.2],
         [0.05, 0.2, 0.05]],
        
        [[0.05, 0.2, 0.05],
         [0.2, -1, 0.2],
         [0.05, 0.2, 0.05]]
    ])

    return signal.convolve(Z, laplacian_kernel, mode='same')

# Store each frame of the simulation
frames_V = []
frames_F = []
frames_k = []

# Run the simulation
for i in range(2000):  # 2000 steps for computational efficiency
    # Compute F and k
    x, y, z = np.indices(U.shape)
    t = np.full(U.shape, i)  # Expand i to match the shape of indices

    F = compute_F(F_init, x, y, z, t)
    k = compute_k(k_init, x, y, z, t)

    # Update U and V with the constraint on surface area
    U_new = U + dt * (Du * laplacian(U) - U*V*V + F*(1-U))
    V_new = V + dt * (Dv * laplacian(V) + U*V*V - (F+k)*V)

    # Clip values to keep within the range of 0 and 1
    U_new = np.clip(U_new, 0, 1)
    V_new = np.clip(V_new, 0, 1)

    # Update U and V
    U, V = U_new, V_new

    if i % 100 == 0:  # Store every 100th frame
        frames_V.append(V.copy())
        frames_F.append(F.copy())
        frames_k.append(k.copy())

# Function to display a frame
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
plt.close()

def display_frame(i):
    fig.clear()
    
    ax1 = fig.add_subplot(131, projection='3d')
    ax1.voxels(frames_V[i] > 0.2, facecolors='b', alpha=0.2)
    ax1.set_title('Frame: %d' % i)

    ax2 = fig.add_subplot(132, projection='3d')
    ax2.scatter(*np.indices(frames_F[i].shape), c=frames_F[i].ravel(), alpha=0.5)
    ax2.set_title('F Distribution')

    ax3 = fig.add_subplot(133, projection='3d')
    ax3.scatter(*np.indices(frames_k[i].shape), c=frames_k[i].ravel(), alpha=0.5)
    ax3.set_title('k Distribution')

    display(fig)

# Use a slider to navigate through the frames
interactive_plot = interact(display_frame, i=(0, len(frames_V)-1))


interactive(children=(IntSlider(value=9, description='i', max=19), Output()), _dom_classes=('widget-interact',…